# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **<br>
The target values are *yes* and *no*. Clearly students are being *classified* into two categories. Hence this is a classification problem. Regression involves target values which are continuous unlike categorisation which involves discrete target values, like in this case.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [7]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [8]:
from sklearn.metrics import f1_score# TODO: Calculate number of students
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == 'no'].shape[0]

# TODO: Calculate graduation rate
grad_rate = n_passed / float(n_students) 

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate*100)

Student data read successfully!
Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [9]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()
print "\nTarget values:"
print y_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [10]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [11]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, train_size=num_train, random_state=42)
# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
1. **Gaussian Naive Bayes**: Probabilistic classifier based on Bayes' theorem with strong independence assumptions within features.
    * General Applications: 
        1. Used widely as a text classifier to classify documents into one category or another using word frequencies as the features.
        2. Gender Classification
        3. Sentimental Analysis of Text
    * Advantages: 
        1. Highly Scalable
        2. Linear Learning Time
    * Disadvantages:
        1. Assuming Strong Independence among features
    * Why it is suitable?
        * In my opinion most of the features shown in this data set are independent of each other and hence Naive Bayes can be applied. It is efficient in learning as well as predicting and uses **probability** for predicition which makes it pretty accurate.<br><br>
2. **Support Vector Machines**: SVM algorithms are non probabilistic binary linear classifier algorithms. In addition to this SVMs can be used for non linear classification as well using the Kernel Trick.
    * General Applications: 
        1. Classification of images can also be performed using SVMs. Experimental results show that SVMs achieve significantly higher search accuracy than traditional query refinement schemes after just three to four rounds of relevance feedback.
        2. Hand-written characters can be recognized using SVM.
        3. The SVM algorithm has been widely applied in the biological and other sciences. They have been used to classify proteins with up to 90% of the compounds classified correctly. 
        (https://en.wikipedia.org/wiki/Support_vector_machine)
    * Advantages: 
        1. Robust model because of maximization of margin.
        2. Supports kernels which is a big advantage against simple linear classification. Kernels are supported by a few other models as well but SVM is kind of based on kernels so using kernels is easier with SVM.
    * Disadvantages: 
        1. Requires full labeling of input data
        2. Uncalibrated class membership probabilities
        3. Parameters of a solved model are difficult to interpret
        4. Space Complexity O(n^2) and time complexity O(n^3)
    * Why it is suitable?
        * It is a binary classifier supporting finite dimensions(using Kernels) so I believe it can accurately make predictions. The data we want to classify here is also binary in nature (Passed or Failed).<br><br>
3. **Random Forest Classifier**: Combines random decision trees with bagging to achieve high classification accuracy. 
    * General Applications: 
        1. Internet traffic interception - certain governments (possibly from the middle east) would like to restrict certain categories of web pages. For example, due to religious restrictions, certain movie pages may be restricted/censored. 
        2.  Video classification - as and when you upload a video on youtube, the video has to be classified into appropriate categories and meta-data added to it.
        3. Image classification - classic problem in image processing. Is this an image of a dog or a cat?
        4. Voice classification: classify and if possible, identify the voice. Very useful in Siri type applications.<br>
***Source : [What are the interesting applications of classification algorithms like neural networks and random forests?](https://www.quora.com/What-are-the-interesting-applications-of-classification-algorithms-like-neural-networks-and-random-forests)***
    * Advantages: 
        1. Do not expect linear features or even features that interact linearly
        2. Handle high dimensional spaces as well as large number of training examples, very well.<br>
        ***Source: [What are the advantages of different classification algorithms?](https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms)***
        3. Fast run times and quite scalable. Space and learning complexity are both roughly O(NlogN) where N is the number of elements in the dataset.
    * Disadvantages: 
        1. This algorithm may overfit to the training data. If the decision trees used in each iteration overfit to a subset of the data, we may end up getting zero error every time which ultimately will result in overfitting of the data.
    * Why it is suitable? 
        * It is an ensemble of decision trees and may result in a very accurate model.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [12]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [13]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.naive_bayes import GaussianNB
# from sklearn import model_B
from sklearn.svm import SVC
# from skearln import model_C
from sklearn.ensemble import RandomForestClassifier
# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = SVC(random_state=42)
clf_C = RandomForestClassifier(random_state=42)
for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: \n".format(clf.__class__.__name__)
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


GaussianNB: 

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8467.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.8029.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8406.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7244.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8038.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7634.

SVC: 

Training a SVC using a training set size of 100. . .
Trained model in 0.0360 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8777.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7746.
Training a SVC using a training set size of 200. . .
Trained mod

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Gaussian Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0010       |        0.0000          |        0.8467    |      0.8029     |
| 200               |            0.0010       |        0.0010          |        0.8406    |      0.7244     |
| 300               |            0.0020       |        0.0010          |        0.8038    |      0.7634     | 

** Classifer 2 - Support Vector Machine SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0360       |        0.0010          |        0.8777    |        0.7746   |
| 200               |            0.0040       |        0.0020          |        0.8679    |        0.7815   |
| 300               |            0.0090       |        0.0020          |        0.8761    |        0.7838   |

** Classifer 3 - Random Forest Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0480       |            0.0010      |        0.9844    |        0.7153   |
| 200               |            0.0260       |            0.0020      |        0.9964    |        0.7857   |
| 300               |            0.0290       |            0.0010      |        0.9951    |        0.8092   |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **<br>
Out of the above three models we can clearly see that the Gaussian Naive Bayes has the least training and prediction times, significantly lower than the Random Forest Classsifier and moderately lower than the SVC. <br>
We can also notice that the F1 scores are consistently higher for the Support Vector Machine (except the Random Forest Classifier score on Training set which probably shows that the RFC is overfitting to the training data) and in a general trend, appear to be increasing with increasing data size.<br>
Therefore, it is safe to say that I choose the **Support Vector Machine SVC** model as the best model considering all physical factors. Granted that SVMs have a quadratic space and cubic time complexity respectively but in a general trend a properly trained SVM can result in a consistently high F1 Score. Considering we need to reach a graduation rate of 95%, the extra processing/prediction is negligible. An SVM can be tweaked in many ways to increase the F1 Score like tweaking the kernel for better classification. Usage of Kernels is one of the main advantages of the SVM.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **<br>
Support Vector Machine are a set of machine learning models which enable us to classify our data into one or another group. It can thus be called a binary classifier. Binary simply because we have two resultant categories.
The image below shows how an SVM works by drawing a clear boundary between the data to classify it into two groups.
![http://www.mblondel.org/images/svm_linear.png](http://www.mblondel.org/images/svm_linear.png) 
<center>Image Source: <a href='http://www.mblondel.org/images/svm_linear.png'> http://www.mblondel.org/images/svm_linear.png</a></center>
<br>
SVM tries to find or 'fit' a somewhat clear boundary between the categories of the data set. In our case, we are training the SVM by supplying data about past students. The result is a function that can draw a boundary between the two categories of data.
Imagine you have a neatly arranged ball pit in front of you. Now SVM is basically a mechanism that positions the partition such that there is maximum space on both the sides. By virtue of the SVM you can put new balls into the pit even when you are not looking(something like blindfolded). You give a ball to the SVM and it can determine which side of the partition it needs to go simply based on how the previous balls were arranged.
![https://udacity-github-sync-content.s3.amazonaws.com/_imgs/8650/1461641602/svmFeatureSpace.gif](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/8650/1461641602/svmFeatureSpace.gif) 
<center>Image Source: <a href='https://udacity-github-sync-content.s3.amazonaws.com/_imgs/8650/1461641602/svmFeatureSpace.gif'> https://udacity-github-sync-content.s3.amazonaws.com/_imgs/8650/1461641602/svmFeatureSpace.gif</a></center>
In the cases where the boundary isn't appearing to be linear or it is difficult to fnid a linear boundary, SVMs support **Kernels** which allow you to draw a boundary in practically any geomteric shape. 
Consider the ball pit example. We are restricted to a partition which is in the shape of a line. However in case the balls are initially arranged in a not so organised manner. In that case, we can somehow raise one type of balls to a height while keeping the others at the same place. Now the SVM can categorise the balls by their height. This is roughly what a kernel does. It allows you to imagine more dimensions for your data then there originally are.
<br>
![https://udacity-github-sync-content.s3.amazonaws.com/_imgs/19273/1464624531/1gvce.png](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/19273/1464624531/1gvce.png)
<center>Image Source: <a href='https://udacity-github-sync-content.s3.amazonaws.com/_imgs/19273/1464624531/1gvce.png'> https://udacity-github-sync-content.s3.amazonaws.com/_imgs/19273/1464624531/1gvce.png</a></center>
<br>
The Kernel Trick makes the SVMs even more powerful and versatile. These soft margins/boundaries are also called **hyperplanes**
SVM accuracy can be measured using F1 scores. A higher F1 Score means a better trained model. 
<br><br>
***Source : [Wikipedia](https://en.wikipedia.org/wiki/Support_vector_machine)***

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [14]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
# TODO: Create the parameters list you wish to tune
parameters = {'C': [1, 10, 100, 200, 300, 400, 500, 1000],
              'gamma': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]}
# cv = StratifiedShuffleSplit(y_all, random_state=42)
# TODO: Initialize the classifier
clf = SVC(kernel='rbf')

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, cv=10, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0050 seconds.
Tuned model has a training F1 score of 0.8462.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.8028.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **<br>
The final model's F1 score for training and testing are **0.8462** and **0.8028** respectively. The kernel used was RBF. (NOTE: I tried to vary the kernels in the Grid Search as well but it seemed like it was going into an infinite loop of some sort.) <br>
The testing score has definitely increased which is a good sign. The training score on the other hand has decreased marginally. This could mean that the optimal model is less overfitting than the original model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.